# Assembly: functions
_COSC 208, Introduction to Computer Systems, Spring 2025_

## Function calls in assembly

* Calling conventions
    * _In which registers are parameters stored?_ — `x0`/`w0`, `x1`/`w1`, `x2`/`w2`, ...
    * _In which register is the return value stored?_ — `x0`/`w0`
    * Return address stored in `x30`
    * Caller's return address stored in callee's stack frame
* Use `bl` (branch & link) to make a function call
    * Store `pc+4` in `x30`
    * Update `pc` to specified code address

_Example_

In [ ]:
/*1*/  int helper(int a) {
/*2*/      a = a + 1;
/*3*/      return a;
/*4*/  }
/*5*/  int main() {
/*6*/      int b = helper(2);
/*7*/      return b;
/*8*/  }

```
0000000000000714 <helper>:
 714:   d10043ff    sub sp, sp, #0x10    // 1
 718:   b9000fe0    str w0, [sp, #12]    // 1
 71c:   b9400fe0    ldr w0, [sp, #12]    // 2
 720:   11000400    add w0, w0, #0x1     // 2
 724:   b9000fe0    str w0, [sp, #12]    // 2
 728:   b9400fe0    ldr w0, [sp, #12]    // 3
 72c:   910043ff    add sp, sp, #0x10    // 3
 730:   d65f03c0    ret                  // 3
```

```
0000000000000734 <main>:
 734:   d10043ff    sub sp, sp, #0x10    // 5
 738:   f81e0ffe    str x30, [sp, #8]    // 5
 73c:   52800040    mov w0, #0x2         // 6
 740:   97fffff6    bl  714 <helper>     // 6
 744:   b9001fe0    str w0, [sp, #4]     // 6
 748:   b9401fe0    ldr w0, [sp, #4]     // 7
 74c:   f84207fe    ldr x30, [sp, #8]    // 7
 750:   910043ff    add sp, sp, #0x10    // 7
 754:   d65f03c0    ret                  // 7
```

```
0000000000000714 <helper>:
 714:   d10043ff    sub sp, sp, #0x10
 718:   b9000fe0    str w0, [sp, #12]
 71c:   b9400fe0    ldr w0, [sp, #12]
 720:   11000400    add w0, w0, #0x1
 724:   b9000fe0    str w0, [sp, #12]
 728:   b9400fe0    ldr w0, [sp, #12]
 72c:   910043ff    add sp, sp, #0x10
 730:   d65f03c0    ret
```

```
0000000000000734 <main>:
 734:   d10043ff    sub sp, sp, #0x10
 738:   f81e0ffe    str x30, [sp, #8]
 73c:   52800040    mov w0, #0x2
 740:   97fffff6    bl  714 <helper>
 744:   b9001fe0    str w0, [sp, #4]
 748:   b9401fe0    ldr w0, [sp, #4]
 74c:   f84207fe    ldr x30, [sp, #8]
 750:   910043ff    add sp, sp, #0x10
 754:   d65f03c0    ret
```

_Assume the registers have the following initial values:_
* `pc` = `0x734`
* `sp` = `0xA80`
* `x30` = `0x600`

```
        0  1  2  3  4  5  6  7  8          
        +--+--+--+--+--+--+--+--+
 0xA60  |                       |
        +--+--+--+--+--+--+--+--+
 0xA68  |           |     3     |
        +--+--+--+--+--+--+--+--+
 0xA70  |           |     3     |
        +--+--+--+--+--+--+--+--+
 0xA78  |        0x600          |
        +--+--+--+--+--+--+--+--+
sp = 0xA80
pc = 0x600
w/x0 = 3
```

<div style="height:1em;"></div>

_The following C code was compiled into assembly (using `clang`)._

In [ ]:
/*1*/  int multiply(int a, int b) {
/*2*/      int c = a * b;
/*3*/      return c;
/*4*/  }
/*5*/  int volume(int x, int y, int z) {
/*6*/      int w = multiply(x, y);
/*7*/      w = multiply(w, z);
/*8*/      return w;
/*9*/  }
/*10*/  int main() {
/*11*/      int n = volume(1, 3, 5);
/*12*/      return n;
/*13*/  }

Q1: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._ 

```
0000000000000544 <multiply>:
  544:    d10043ff     sub sp, sp, #0x10    // 1
  548:    b9000fe0     str w0, [sp, #12]    // 1
  54c:    b9000be1     str w1, [sp, #8]     // 1
  550:    b9400fe8     ldr w8, [sp, #12]    // 2
  554:    b9400be9     ldr w9, [sp, #8]     // 2
  558:    1b097d08     mul w8, w8, w9       // 2
  55c:    b90007e8     str w8, [sp, #4]     // 2
  560:    b94007e0     ldr w0, [sp, #4]     // 3
  564:    910043ff     add sp, sp, #0x10    // 3
  568:    d65f03c0     ret                  // 3
```

```
000000000000056c <volume>:
  56c:    d10083ff     sub sp, sp, #0x20    // 5
  570:    f9000bfe     str x30, [sp, #16]   // 5
  574:    b9000fe0     str w0, [sp, #12]    // 5
  578:    b9000be1     str w1, [sp, #8]     // 5
  57c:    b90007e2     str w2, [sp, #4]     // 5
  580:    b9400fe0     ldr w0, [sp, #12]    // 6
  584:    b9400be1     ldr w1, [sp, #8]     // 6
  588:    97ffffef     bl  544 <multiply>   // 6
  58c:    b90003e0     str w0, [sp]         // 6
  590:    b94003e0     ldr w0, [sp]         // 7
  594:    b94007e1     ldr w1, [sp, #4]     // 7
  598:    97ffffeb     bl  544 <multiply>   // 7
  59c:    b90003e0     str w0, [sp]         // 7
  5a0:    b94003e0     ldr w0, [sp]         // 8
  5a4:    f9400bfe     ldr x30, [sp, #16]   // 8
  5a8:    910083ff     add sp, sp, #0x20    // 8
  5ac:    d65f03c0     ret                  // 8
```

```
00000000000005b0 <main>:                               
  5b0:    d10083ff     sub sp, sp, #0x20     // 10
  5b4:    f9000bfe     str x30, [sp, #16]    // 10
  5b8:    52800020     mov w0, #0x1          // 11
  5bc:    52800061     mov w1, #0x3          // 11
  5c0:    528000a2     mov w2, #0x5          // 11
  5c4:    97ffffe9     bl  56c <volume>      // 11
  5c8:    b9000be0     str w0, [sp, #8]      // 11
  5cc:    b9400be0     ldr w0, [sp, #8]      // 12
  5d0:    f9400bfe     ldr x30, [sp, #16]    // 12
  5d4:    910083ff     add sp, sp, #0x20     // 12
  5d8:    d65f03c0     ret                   // 12
```

```
0000000000000544 <multiply>:
  544:    d10043ff     sub sp, sp, #0x10
  548:    b9000fe0     str w0, [sp, #12]
  54c:    b9000be1     str w1, [sp, #8]
  550:    b9400fe8     ldr w8, [sp, #12]
  554:    b9400be9     ldr w9, [sp, #8]
  558:    1b097d08     mul w8, w8, w9
  55c:    b90007e8     str w8, [sp, #4]
  560:    b94007e0     ldr w0, [sp, #4]
  564:    910043ff     add sp, sp, #0x10
  568:    d65f03c0     ret
```

```
000000000000056c <volume>:
  56c:    d10083ff     sub sp, sp, #0x20
  570:    f9000bfe     str x30, [sp, #16]
  574:    b9000fe0     str w0, [sp, #12]
  578:    b9000be1     str w1, [sp, #8]
  57c:    b90007e2     str w2, [sp, #4]
  580:    b9400fe0     ldr w0, [sp, #12]
  584:    b9400be1     ldr w1, [sp, #8]
  588:    97ffffef     bl  544 <multiply>
  58c:    b90003e0     str w0, [sp]
  590:    b94003e0     ldr w0, [sp]
  594:    b94007e1     ldr w1, [sp, #4]
  598:    97ffffeb     bl  544 <multiply>
  59c:    b90003e0     str w0, [sp]
  5a0:    b94003e0     ldr w0, [sp]
  5a4:    f9400bfe     ldr x30, [sp, #16]
  5a8:    910083ff     add sp, sp, #0x20
  5ac:    d65f03c0     ret
```

```
00000000000005b0 <main>:                               
  5b0:    d10083ff     sub sp, sp, #0x20
  5b4:    f9000bfe     str x30, [sp, #16]
  5b8:    52800020     mov w0, #0x1
  5bc:    52800061     mov w1, #0x3
  5c0:    528000a2     mov w2, #0x5
  5c4:    97ffffe9     bl  56c <volume>
  5c8:    b9000be0     str w0, [sp, #8]
  5cc:    b9400be0     ldr w0, [sp, #8]
  5d0:    f9400bfe     ldr x30, [sp, #16]
  5d4:    910083ff     add sp, sp, #0x20
  5d8:    d65f03c0     ret
```

Q2: _Assume the registers initially hold the following values:_
```
pc = 0x5B0
sp = 0xFF0
w/x30 = 0x500
```
_Draw the contents of the stack and registers after executing the assembly instruction at **0x58C**._

[Video demonstrating how to solve this problem](https://colgate.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=53efda4c-c9b2-4959-937b-b22101009cd2)

<iframe src="https://colgate.hosted.panopto.com/Panopto/Pages/Embed.aspx?id=53efda4c-c9b2-4959-937b-b22101009cd2&autoplay=false&offerviewer=true&showtitle=true&showbrand=true&captions=false&interactivity=all" height="405" width="720" style="border: 1px solid #464646;" allowfullscreen allow="autoplay" aria-label="Panopto Embedded Video Player" aria-description="COSC 208: Assembly: Functions Q2 Solution" ></iframe>

<div style="height:38em;"></div>

_The following C code was compiled into assembly (using `clang`)._

In [ ]:
/* 1*/  int sum(int a, int b) {
/* 2*/      int c = a + b;
/* 3*/      return c;
/* 4*/  }
/* 5*/  int triple(int u, int v) {
/* 6*/      int r = 3;
/* 7*/      int s = sum(u, v);
/* 8*/      int t = s * r;
/* 9*/      return t;
/*10*/  }
/*11*/  int main() {
/*12*/      int x = 2;
/*13*/      int y = 3;
/*14*/      int z = triple(x, y);
/*15*/      return z;
/*16*/  }

Q3: _For each line of assembly, indicate which original line of C code the instruction was derived from._

```
0000000000400544 <sum>:
    400544:    d10043ff     sub    sp, sp, #0x10   // 1 
    400548:    b9000fe0     str    w0, [sp, #12]   // 1
    40054c:    b9000be1     str    w1, [sp, #8]    // 1
    400550:    b9400fe8     ldr    w8, [sp, #12]   // 2
    400554:    b9400be9     ldr    w9, [sp, #8]    // 2
    400558:    0b090108     add    w8, w8, w9      // 2
    40055c:    b90007e8     str    w8, [sp, #4]    // 2
    400560:    b94007e0     ldr    w0, [sp, #4]    // 3
    400564:    910043ff     add    sp, sp, #0x10   // 3
    400568:    d65f03c0     ret                    // 3
```

```
000000000040056c <triple>:
    40056c:    d100c3ff     sub    sp, sp, #0x30   // 5
    400570:    f90013fe     str    x30, [sp, #32]  // 5
    400574:    b9001fe0     str    w0, [sp, #28]   // 5
    400578:    b9001be1     str    w1, [sp, #24]   // 5
    40057c:    52800068     mov    w8, #0x3        // 6
    400580:    b90017e8     str    w8, [sp, #20]   // 6
    400584:    b9401fe0     ldr    w0, [sp, #28]   // 7
    400588:    b9401be1     ldr    w1, [sp, #24]   // 7
    40058c:    97ffffee     bl     400544 <sum>    // 7
    400590:    b90013e0     str    w0, [sp, #16]   // 7
    400594:    b94013e8     ldr    w8, [sp, #16]   // 8
    400598:    b94017e9     ldr    w9, [sp, #20]   // 8
    40059c:    1b097d08     mul    w8, w8, w9      // 8
    4005a0:    b9000fe8     str    w8, [sp, #12]   // 8
    4005a4:    b9400fe0     ldr    w0, [sp, #12]   // 9
    4005a8:    f94013fe     ldr    x30, [sp, #32]  // 9
    4005ac:    9100c3ff     add    sp, sp, #0x30   // 9
    4005b0:    d65f03c0     ret                    // 9
```

```
00000000004005b4 <main>:
    4005b4:    d10083ff     sub    sp, sp, #0x20   // 11
    4005b8:    f9000bfe     str    x30, [sp, #16]  // 11
    4005bc:    52800048     mov    w8, #0x2        // 12
    4005c0:    b9000be8     str    w8, [sp, #8]    // 12
    4005c4:    52800069     mov    w9, #0x3        // 13
    4005c8:    b90007e9     str    w9, [sp, #4]    // 13
    4005cc:    b9400be0     ldr    w0, [sp, #8]    // 14
    4005d0:    b94007e1     ldr    w1, [sp, #4]    // 14
    4005d4:    97ffffe5     bl    410056c <triple> // 14
    4005d8:    b90003e0     str    w0, [sp]        // 14
    4005dc:    b94003e0     ldr    w0, [sp]        // 15
    4005e0:    f9400bfe     ldr    x30, [sp, #16]  // 15
    4005e4:    910083ff     add    sp, sp, #0x20   // 15
    4005e8:    d65f03c0     ret                    // 15
```

```
0000000000400544 <sum>:
    400544:    d10043ff     sub    sp, sp, #0x10   // 
    400548:    b9000fe0     str    w0, [sp, #12]   // 
    40054c:    b9000be1     str    w1, [sp, #8]    // 
    400550:    b9400fe8     ldr    w8, [sp, #12]   // 
    400554:    b9400be9     ldr    w9, [sp, #8]    // 
    400558:    0b090108     add    w8, w8, w9      // 
    40055c:    b90007e8     str    w8, [sp, #4]    // 
    400560:    b94007e0     ldr    w0, [sp, #4]    // 
    400564:    910043ff     add    sp, sp, #0x10   // 
    400568:    d65f03c0     ret                    // 
```

```
000000000040056c <triple>:
    40056c:    d100c3ff     sub    sp, sp, #0x30   // 
    400570:    f90013fe     str    x30, [sp, #32]  // 
    400574:    b9001fe0     str    w0, [sp, #28]   // 
    400578:    b9001be1     str    w1, [sp, #24]   // 
    40057c:    52800068     mov    w8, #0x3        // 
    400580:    b90017e8     str    w8, [sp, #20]   // 
    400584:    b9401fe0     ldr    w0, [sp, #28]   // 
    400588:    b9401be1     ldr    w1, [sp, #24]   // 
    40058c:    97ffffee     bl     400544 <sum>    //  
    400590:    b90013e0     str    w0, [sp, #16]   // 
    400594:    b94013e8     ldr    w8, [sp, #16]   // 
    400598:    b94017e9     ldr    w9, [sp, #20]   // 
    40059c:    1b097d08     mul    w8, w8, w9      // 
    4005a0:    b9000fe8     str    w8, [sp, #12]   // 
```

```
    4005a4:    b9400fe0     ldr    w0, [sp, #12]   // 
    4005a8:    f94013fe     ldr    x30, [sp, #32]  // 
    4005ac:    9100c3ff     add    sp, sp, #0x30   // 
    4005b0:    d65f03c0     ret                    // 
```

```
00000000004005b4 <main>:
    4005b4:    d10083ff     sub    sp, sp, #0x20   // 
    4005b8:    f9000bfe     str    x30, [sp, #16]  // 
    4005bc:    52800048     mov    w8, #0x2        // 
    4005c0:    b9000be8     str    w8, [sp, #8]    // 
    4005c4:    52800069     mov    w9, #0x3        // 
    4005c8:    b90007e9     str    w9, [sp, #4]    // 
    4005cc:    b9400be0     ldr    w0, [sp, #8]    // 
    4005d0:    b94007e1     ldr    w1, [sp, #4]    // 
    4005d4:    97ffffe5     bl     40056c <triple> //  
    4005d8:    b90003e0     str    w0, [sp]        // 
    4005dc:    b94003e0     ldr    w0, [sp]        // 
    4005e0:    f9400bfe     ldr    x30, [sp, #16]  // 
    4005e4:    910083ff     add    sp, sp, #0x20   // 
    4005e8:    d65f03c0     ret                    // 
```

Q4: _Assume the registers have the following initial values:_

* `pc` = `0x4005B4` 
* `sp` = `0xFF0`
* `x30` = `0x400488`
 
_Draw the contents of the stack and registers after executing the assembly instruction at **0x4005AC**._

<div style="height:28em;"></div>

_The following C code was compiled into assembly (using `clang`)._

In [ ]:
/* 1*/  int interest_due(int outstanding, int rate) {
/* 2*/      int divisor = 1200;
/* 3*/      int annual = outstanding * rate;
/* 4*/      int monthly = annual / divisor;
/* 5*/      return monthly;
/* 6*/  }
/* 7*/
/* 8*/  int make_payment(int outstanding, int payment, int rate) {
/* 9*/      int interest = interest_due(outstanding, rate);
/*10*/      int principal = payment - interest;
/*11*/      if (principal > outstanding) {
/*12*/          outstanding = 0;
/*13*/      } else {
/*14*/          outstanding -= principal;    
/*15*/      }
/*16*/      return outstanding;
/*17*/  }

Q5: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

```
00000000004005d4 <interest_due>:
  4005d4:    d10083ff     sub    sp, sp, #0x20   // 1
  4005d8:    b9001fe0     str    w0, [sp, #28]   // 1
  4005dc:    b9001be1     str    w1, [sp, #24]   // 1
  4005e0:    52809608     mov    w8, #0x4b0      // 2
  4005e4:    b90017e8     str    w8, [sp, #20]   // 2
  4005e8:    b9401fe8     ldr    w8, [sp, #28]   // 3
  4005ec:    b9401be9     ldr    w9, [sp, #24]   // 3
  4005f0:    1b097d08     mul    w8, w8, w9      // 3
  4005f4:    b90013e8     str    w8, [sp, #16]   // 3
  4005f8:    b94013e8     ldr    w8, [sp, #16]   // 4
  4005fc:    b94017e9     ldr    w9, [sp, #20]   // 4
  400600:    1ac90d08     sdiv   w8, w8, w9      // 4
  400604:    b9000fe8     str    w8, [sp, #12]   // 4
  400608:    b9400fe0     ldr    w0, [sp, #12]   // 5
  40060c:    910083ff     add    sp, sp, #0x20   // 5
  400610:    d65f03c0     ret                    // 5
```

```
0000000000400614 <make_payment>:                 
  400614:    d100c3ff     sub    sp, sp, #0x30   // 8
  400618:    f90013fe     str    x30, [sp, #32]  // 8
  40061c:    b9001fe0     str    w0, [sp, #28]   // 8
  400620:    b9001be1     str    w1, [sp, #24]   // 8
  400624:    b90017e2     str    w2, [sp, #20]   // 8
  400628:    b9401fe0     ldr    w0, [sp, #28]   // 9
  40062c:    b94017e1     ldr    w1, [sp, #20]   // 9
  400630:    97ffffe9     bl     4005d4 <interest_due> // 9
  400634:    b90013e0     str    w0, [sp, #16]   // 9
  400638:    b9401be8     ldr    w8, [sp, #24]   // 10
  40063c:    b94013e9     ldr    w9, [sp, #16]   // 10
  400640:    6b090108     subs    w8, w8, w9     // 10
  400644:    b9000fe8     str    w8, [sp, #12]   // 10
  400648:    b9400fe8     ldr    w8, [sp, #12]   // 11
  40064c:    b9401fe9     ldr    w9, [sp, #28]   // 11
  400650:    6b09011f     cmp    w8, w9          // 11
  400654:    5400006d     b.le   400660 <make_payment+0x4c>  // 11
  400658:    b9001fff     str    wzr, [sp, #28]  // 12
  40065c:    14000005     b      400670 <make_payment+0x5c>  // 13
  400660:    b9400fe8     ldr    w8, [sp, #12]   // 14
  400664:    b9401fe9     ldr    w9, [sp, #28]   // 14
  400668:    6b080128     subs   w8, w9, w8      // 14
  40066c:    b9001fe8     str    w8, [sp, #28]   // 14
  400670:    b9401fe0     ldr    w0, [sp, #28]   // 16
  400674:    f94013fe     ldr    x30, [sp, #32]  // 16
  400678:    9100c3ff     add    sp, sp, #0x30   // 16
  40067c:    d65f03c0     ret                    // 16
```

```
00000000004005d4 <interest_due>:
  4005d4:    d10083ff     sub    sp, sp, #0x20  //
  4005d8:    b9001fe0     str    w0, [sp, #28]  //
  4005dc:    b9001be1     str    w1, [sp, #24]  //
  4005e0:    52809608     mov    w8, #0x4b0     //
  4005e4:    b90017e8     str    w8, [sp, #20]  //
  4005e8:    b9401fe8     ldr    w8, [sp, #28]  //
  4005ec:    b9401be9     ldr    w9, [sp, #24]  //
  4005f0:    1b097d08     mul    w8, w8, w9     //
  4005f4:    b90013e8     str    w8, [sp, #16]  //
  4005f8:    b94013e8     ldr    w8, [sp, #16]  //
  4005fc:    b94017e9     ldr    w9, [sp, #20]  //
  400600:    1ac90d08     sdiv   w8, w8, w9     //
  400604:    b9000fe8     str    w8, [sp, #12]  //
  400608:    b9400fe0     ldr    w0, [sp, #12]  //
  40060c:    910083ff     add    sp, sp, #0x20  //
  400610:    d65f03c0     ret                   //
```

```
0000000000400614 <make_payment>:
  400614:    d100c3ff     sub    sp, sp, #0x30   //
  400618:    f90013fe     str    x30, [sp, #32]  //
  40061c:    b9001fe0     str    w0, [sp, #28]   //
  400620:    b9001be1     str    w1, [sp, #24]   //
  400624:    b90017e2     str    w2, [sp, #20]   //
  400628:    b9401fe0     ldr    w0, [sp, #28]   //
  40062c:    b94017e1     ldr    w1, [sp, #20]   //
  400630:    97ffffe9     bl     4005d4 <interest_due>  //
  400634:    b90013e0     str    w0, [sp, #16]   //
```

```
  400638:    b9401be8     ldr    w8, [sp, #24]   //
  40063c:    b94013e9     ldr    w9, [sp, #16]   //
  400640:    6b090108     subs   w8, w8, w9      //
  400644:    b9000fe8     str    w8, [sp, #12]   //
  400648:    b9400fe8     ldr    w8, [sp, #12]   //
  40064c:    b9401fe9     ldr    w9, [sp, #28]   //
  400650:    6b09011f     cmp    w8, w9          //
  400654:    5400006d     b.le   400660 <make_payment+0x4c>  //
  400658:    b9001fff     str    wzr, [sp, #28]  //
  40065c:    14000005     b      400670 <make_payment+0x5c>  //
  400660:    b9400fe8     ldr    w8, [sp, #12]   //
  400664:    b9401fe9     ldr    w9, [sp, #28]   //
  400668:    6b080128     subs   w8, w9, w8      //
  40066c:    b9001fe8     str    w8, [sp, #28]   //
  400670:    b9401fe0     ldr    w0, [sp, #28]   //
  400674:    f94013fe     ldr    x30, [sp, #32]  //
  400678:    9100c3ff     add    sp, sp, #0x30   //
  40067c:    d65f03c0     ret                    //
```

Q6: _Assume the initial values of the registers are as follows:_
* `pc = 0x400614`
* `sp = 0xf80`
* `w0 = 100000`
* `w1 = 500`
* `w2 = 3`
* `x30 = 0x400400`

_Draw the contents of the stack and registers after executing the assembly instruction at **0x40060C**._

Final register values:
* `pc = 0x400610`
* `sp = 0xF50`
* `w/x0 = 250`
* `w/x1 = 3`
* `w/x2 = 3`
* `w/x8 = 250`
* `w/x9 = 0x4B0`
* `x30 = 0x400634`

Final stack:
```
      ┌─────────────────┐   ──╮
0xF30 │                 │     │
      ├────────┬────────┤     │
0xF38 │        │  250   │     │
      ├────────┼────────┤     │ months_remain
0xF40 │ 300000 │  0x4B0 │     │
      ├────────┼────────┤     │
0xF48 │   3    │ 100000 │     │
sp ─> ├────────┴────────┤   ──┤
0xF50 │                 │     │
      ├─────────────────┤     │
0xF58 │                 │     │
      ├────────┬────────┤     │
0xF60 │        │   3    │     │
      ├────────┼────────┤     │ make_payment
0xF68 │   500  │ 100000 │     │
      ├────────┴────────┤     │
0xF70 │    0x400400     │     │
      ├─────────────────┤     │
0xF78 │                 │     │
      ├─────────────────┤   ──╯
0xF80 │                 │
      └─────────────────┘
```

<div style="height:25em;"></div>

Q7: _Create a `make_payment_goto` function that behaves the same as the `make_payment` function but uses `goto` statements, just as a compiler would do when generating assembly code._

In [ ]:
void make_payment_goto(int outstanding, int payment, int rate) {
  int interest = interest_due(outstanding, rate);
  int principal = payment - interest);
  if (principal <= outstanding)
    goto ELSE;
  outstanding = 0;
  goto END;
ELSE:
  outstanding == principal;
END:
  return outstanding;
}

<div style="height:15em;"></div>